# Introduction

## Explanation 1.1: Linear Programming

In the offline scenario, the total utility of the optimal task assignment, which is marked in red in TABLE 1, is 17.

|    | t1 | t2 | t3 | t4 | t5 | t6 |
|----|----|----|----|----|----|----|
| w1(1) |  7  |  1   |    |    |    |    |
| w2(3) |    |     |  1  |    |    |    |
| w3(2) |    |    |   9 |  1  |  1  |  1  |

In [15]:
from scipy.optimize import linprog
import numpy as np

c = -np.array([7, 1, 0, 0, 0, 0,
               0, 0, 1, 0, 0, 0,
               0, 0, 9, 1, 1, 1
               ])

A_ub = [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        ]

b_ub = [1, 3, 2, 1, 1, 1, 1, 1, 1]

bounds = [(0, 1) for _ in range(18)]

res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=1)

res.fun

-17.0

In [16]:
res.x

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  1.])

## Explanation 1.2

The GOMA problem, which generalizes the OMWBM problem, mainly differs from the OMWBM problem in that both the tasks and the workers are dynamic. 

| Timestamp | 0  | 1  | 2 | 7 | 8 | 9 | 9 | 15 | 18 |
|-----------|----|----|---|---|---|---|---|----|----|
| 1st order | t1 | t2 | w1  | w2  | t3  | t4  | w3  | t5   | t6   |
| 2nd order | w1 | t1   | t2  | t3  | w3  | t4  | w2  | t6   | t5   |

see Keynote.

# Problem Statement

Dataset Format Explanation:

## Header

| workerN/n | taskN/m  | Umax  | sumC |
|-----------|----|----|---|
| 817 | 4036 | 10.000000 | 4853  | 

## Task

| begTime | stype  | loc.first  | loc.second | endTime | cost.pay |
|-----------|----|----|---|----|---|
| 74875 | t | 1.790979 | 1.725669  |  600 | 3.6|


## Worker

| begTime | stype  | loc.first  | loc.second | rad | cap | endTime | cost.rate |
|-----------|----|----|---|---|---|---|---|
| 444046 | w |1.230049 |1.007416 |1.000000| 1| 600 |0.597888|

In [22]:
from typing import Tuple, Self
import os


class Vertex:
    """Either a task or a worker. """

    def __init__(self, arrival_time: int, location: Tuple[float],  deadline: int) -> None:
        assert len(location) == 2
        self.arrival_time = arrival_time
        self.location = location
        self.deadline = deadline

    def distance_from(self, v: Self):
        x1, y1 = self.location
        x2, y2 = v.location
        return ((x1-x2)**2+(y1-y2)**2)**(1/2)


class Worker(Vertex):
    """A worker is a tuple <L, A, D, R, C, Delta>, who arrives at the initial location L in the 2D space, 
    at time A. He/She can perform several tasks that arrive at the platform before his/her response deadline D
    with several constraints:
    1. He/She can perform at most C tasks (i.e., his/her capacity); 
    2. He/She can only perform tasks posted within a circular range centered at L and with a radius of R;
    3. Delta is the success ratio based on his/her historical task completion records.
    """

    def __init__(self, arrival_time: int, location: Tuple[float], radius: float, capacity: int, deadline: int, success_ratio: float) -> None:
        assert 0 < success_ratio <= 1
        super().__init__(arrival_time, location, deadline)
        self.radius = radius
        self.capacity = capacity
        self.success_ratio = success_ratio

    def __str__(self) -> str:
        return f"w: loc = {self.location}, rad = {self.radius:.2f}, cap = {self.capacity}, time = ({self.arrival_time}, {self.deadline}), success_ratio = {self.success_ratio:.2f}"


class Task(Vertex):
    """A task is a tuple <L, A, D, P>, which is posted at the location L in 2D space, at time A. 
    It can only be allocated to a worker who arrives before the response deadline D, with a payoff P.

    In the given dataset:
    A, D-A are an integer timestamp;
    L is composed by x, y float values;
    P is a float value.
    """

    def __init__(self, arrival_time: int, location: Tuple[float], deadline: int, payoff: float) -> None:
        super().__init__(arrival_time, location, deadline)
        self.payoff = payoff

    def __str__(self) -> str:
        return f"t: loc = {self.location}, time = ({self.arrival_time}, {self.deadline}), payoff = {self.payoff}"

    def utility_performed_by(self, worker: type[Worker]):
        """The utility that a worker performs a task is measured by P_t * Delta_w.
        Without loss of generality, we assume that utility is between 0 and U_max.
        """
        return self.payoff * worker.success_ratio

Based on different assumptions on the arrival order, we introduce two competitive analysis models for the GOMA problem, the adversarial
order model and the random order model, which focus on the
worst-case arrival order and the stochastic arrival order of
all the tasks and workers, respectively.

In [26]:
class GOMA:
    """Given a set of tasks T, a set of workers W, and a utility function U (., .),
    the GOMA problem is to find an allocation M 
    to maximize the total utility such that the following constraints are satisfied:
    1. Deadline Constraint;
    2. Invariable Constraint;
    3. Capacity Constraint;
    4. Range Constraint.
    """

    def __init__(self, filepath: str = "../dataset/real/EverySender_cap1/data_00.txt"):
        assert os.path.exists(filepath)
        pass

    def next(self):
        pass


GOMA()

# Baseline Algorithm: Extended Greedy-RT

**Basic Idea**: first randomly choose a threshold on the weights of edges, 

and then randomly choose an edge incident to each newly arrived vertex among those edges whose weights are *no less than* the threshold.

In [20]:
def algorithm1(problem: type[GOMA]):
    """

    """
    # randomly choose a threshold (ek) on the weights of edges according to the estimated maximum weight Umax

    # When a new vertex arrives, adds an edge among the ones whose weights are no less than a threshold
    # and that satisfy all the constraints to the result

    # If a worker with capacity C arrives, Algorithm 1 regards him/her as duplicates of the same worker with capacity of 1 arrivubg at the same time
    # and processes them one by one

    pass

SyntaxError: incomplete input (2180352630.py, line 10)